En este ejercicio, el alumno tendrá que plantear y resolver un problema utilizando aprendizaje
automático. Se tendrán que llevar a cabo todos los pasos necesarios para la resolución del
problema: planteamiento, diseño, adquisición de los datos, análisis, entrenamiento de
algoritmos, evaluación, discusión de resultados, etc. Todo este proceso se desarrollará en un
cuaderno de Jupyter que conformará la entrega final.
El problema planteado podrá ser inventado o real, y puede estar basado en un problema de
negocio, un artículo científico, una competición, etc. Existen únicamente dos requisitos en
cuanto al caso de uso:
1. Tiene que utilizarse un conjunto de datos para su resolución.
2. La solución debe estar basada en aprendizaje automático, es decir, se debe haber
entrenado al menos un algoritmo.

![nba-logo](images/nba-logo.png)

In [74]:
import pandas as pd 
import numpy as np
from IPython.display import display
import zipfile as zp
import matplotlib.pyplot as plt
import sklearn.preprocessing as prep
from sklearn.metrics import precision_score, recall_score, confusion_matrix, mean_absolute_error, mean_absolute_percentage_error, r2_score, accuracy_score, mean_squared_error

Carga de los dataframes

In [78]:
zf = zp.ZipFile('dataframes/nba.zip')
df_games = pd.read_csv(zf.open('games.csv'))
df_games_details = pd.read_csv(zf.open('games_details.csv'))
df_players = pd.read_csv(zf.open('players.csv'))
df_ranking = pd.read_csv(zf.open('ranking.csv'))
df_teams = pd.read_csv(zf.open('teams.csv'))

Muestra varias filas de cada dataframe para realizar un primer analisis

In [79]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.head())
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.head())
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.head())
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.head())
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.head())

--------- PARTIDOS ----------
-----------------------------


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,0.457,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,0.313,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,0.297,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,0.378,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


--------- DETALLES PARTIDOS ----------
--------------------------------------


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


--------- JUGADORES ----------
------------------------------


,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019


--------- RANKING LIGA ----------
---------------------------------


,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN


--------- EQUIPOS ----------
----------------------------


,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


Muestra filas y columnas de cada dataframe

In [77]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.shape)
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.shape)
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.shape)
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.shape)
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.shape)

--------- PARTIDOS ----------
-----------------------------


(26651, 21)

--------- DETALLES PARTIDOS ----------
--------------------------------------


(668628, 29)

--------- JUGADORES ----------
------------------------------


(7228, 4)

--------- RANKING LIGA ----------
---------------------------------


(210342, 13)

--------- EQUIPOS ----------
----------------------------


(30, 14)